In [1]:
interface BasicDetails {
  readonly id: number;
  name: string;
  level: number;
  days_in_faction: number;
}

interface V1BasicDetails {
  age: number;
  friends: number;
  enemies: number;
}

type Base = BasicDetails & V1BasicDetails
    
interface Member extends Base {
  racing_skill: number;
  total_respect: number;
  best_damage: number;
  total_crimes: number;
  xanax_taken: number;
  energy_refills: number;
  wars_in_faction: number;
  wars_participated: number;
  wars_total_attacks: number;
  wars_total_score: number;
}

const KEYARG = "key=lBSMTY8Q2t3xzIlp"
const NUMDAYS = 90

const now = Temporal.Now.instant()
console.log(now.toLocaleString())
const nowTS = Math.floor(now.epochMilliseconds / 1000)
console.log(nowTS)
const sinceTS = nowTS - NUMDAYS * 24 * 60 * 60
console.log(sinceTS)

20/09/2025, 22:57:32
1758405452
1750629452


In [2]:
const getFactBasic = () => fetch(`https://api.torn.com/v2/faction/basic?${KEYARG}`)

In [3]:
const getFactMembers = () => fetch(`https://api.torn.com/v2/faction/members?striptags=true&${KEYARG}`)

In [4]:
const getV1BasicDetails = (id: number) => fetch(`https://api.torn.com/user/${id}?selections=profile&${KEYARG}`)

In [5]:
const getPersonalStats = (id: number) => fetch(`https://api.torn.com/v2/user/${id}/personalstats?cat=all&${KEYARG}`)

In [6]:
const getRankedWars = () => fetch(`https://api.torn.com/v2/faction/rankedwars?${KEYARG}`)

In [7]:
const getRwReport = (id: number) => fetch(`https://api.torn.com/v2/faction/${id}/rankedwarreport?${KEYARG}`)

In [8]:
const getChains = () => fetch(`https://api.torn.com/v2/faction/chains?${KEYARG}`)

In [9]:
const getChainReport = (id: number) => fetch(`https://api.torn.com/v2/faction/${id}/chainreport?${KEYARG}`)

In [10]:
function createMember(o: BasicDetails): Member {
  return {id: o.id, name: o.name, level: o.level, days_in_faction: o.days_in_faction,
    wars_in_faction: 0, wars_participated: 0, wars_total_attacks: 0, wars_total_score: 0}
}

In [11]:
function updateWithV1BasicDetails(m: Member, o: V1BasicDetails) {
  m.age = o.age
  m.friends = o.friends
  m.enemies = o.enemies
}

In [12]:
function updateWithPersonalStats(m: Member, o: any) {
  m.racing_skill = o.racing.skill
  m.total_respect = o.attacking.faction.respect
  m.best_damage = o.attacking.damage.best
  m.total_crimes = o.crimes.total
  m.xanax_taken = o.drugs.xanax
  m.energy_refills = o.other.refills.energy
}

In [13]:
async function fetchData() {
  const factBasic = await getFactBasic().
      then((resp) => resp.json()).
      then((obj) => obj.basic)
  //console.log(factBasic)
  const factMems = await getFactMembers().
      then((resp) => resp.json()).
      then((obj) => obj.members)
  //console.log(factMems)
  const members: Member[] = factMems.map(createMember)
  for (let m of members) {
    const basic = await getV1BasicDetails(m.id).
        then((resp) => resp.json())
    //console.log(basic)
    updateWithV1BasicDetails(m, basic)
    const stats = await getPersonalStats(m.id).
        then((resp) => resp.json()).
        then((obj) => obj.personalstats)
    //console.log(stats)
    updateWithPersonalStats(m, stats)
  }
  const memberMap = new Map(members.map(m => [m.id, m]))
  //console.log(memberMap)
  const wars = await getRankedWars().
      then((resp) => resp.json()).
      then((obj) => obj.rankedwars)
  //console.log(wars)
  for (const w of wars) {
    if (w.end < sinceTS) {
      break
    }
    const report = await getRwReport(w.id).
        then((resp) => resp.json()).
        then((obj) => obj.rankedwarreport)
    //console.log(report)
    for (const f of report.factions) {
      if (f.id != factBasic.id) {
        continue;
      }
      for (const rwm of f.members) {
        const m = memberMap.get(rwm.id)
        if (m === undefined) {
          //console.log(`Could not find member id ${rwm.id} (${rwm.name}); probably left faction`)
          continue
        }
        m.wars_in_faction++
        if (rwm.attacks > 0) {
          m.wars_participated++
          m.wars_total_attacks += rwm.attacks
          m.wars_total_score += rwm.score
        }
      }
    }
  }
  const chains = await getChains().
      then((resp) => resp.json()).
      then((obj) => obj.chains)
  //console.log(chains)
  for (const ch of chains) {
    if (ch.end < sinceTS || ch.chain < 100) {
        continue
    }
    const report = await getChainReport(ch.id).
        then((resp) => resp.json()).
        then((obj) => obj.chainreport)
    //console.log(report)
    if (report.details.war > 0) {
      // We could extract more war stats, if wanted.
      continue
    }
    for (const att of report.attackers) {
      const m = memberMap.get(att.id)
      if (m === undefined) {
        //console.log(`Could not find member id ${att.id}; probably left faction`)
        continue
      }
      /*m.wars_in_faction++
      if (rwm.attacks > 0) {
        m.wars_participated++
        m.wars_total_attacks += rwm.attacks
        m.wars_total_score += rwm.score
      }*/
    }
  }
  //console.log(memberMap)
  console.log(`# ${NUMDAYS}-day faction member activity report for ${factBasic.name} [${factBasic.id}] upto ${now.toLocaleString()}`) 
  return Array.from(memberMap.values())
}

In [14]:
const data = await fetchData()
//console.log(data)
const fields = Object.keys(data[0])
const replacer = (key, value) => value === null ? '' : value 
var csv = data.map(row => fields.map(fieldName => JSON.stringify(row[fieldName], replacer)).join(','))
csv.unshift(fields.join(',')) // add header row
const out = csv.join('\r\n');
console.log(out)

# 90-day faction member activity report for Human Factor [48025] upto 20/09/2025, 22:57:32
id,name,level,days_in_faction,wars_in_faction,wars_participated,wars_total_attacks,wars_total_score,age,friends,enemies,racing_skill,total_respect,best_damage,total_crimes,xanax_taken,energy_refills
1513734,"Big_Tex1429",93,270,3,3,213,1407.09,5406,20,10,16,42273,7231,52369,2776,1316
2122084,"Mingle",100,277,3,3,401,2548.84,2687,57,18,32,59132,17599,78399,3102,1030
2143613,"Talendrife",85,277,3,3,260,1646.1,2646,7,5,25,70717,13149,106583,54,1866
2637832,"Lexii",90,277,3,3,306,2576.55,1727,114,18,28,44585,8694,77828,3044,986
2733982,"DukeSilver",84,16,1,1,81,481.2,1398,21,26,7,18499,8075,46593,2147,636
2734960,"Sofro",24,67,2,2,112,703.66,1394,1,0,4,1497,1673,4450,28,97
2801249,"Socharis",96,271,3,2,232,1446.25,1159,9,1,50,45968,5602,70635,1887,847
2843077,"AUSTINEZ",29,178,3,2,40,236.23,1049,4,4,3,2442,1794,3029,115,21
2919031,"Rhysand",55,271,3,2,153,697.49,880,8,3,6,11012,3210,42012,1314,218
30